# Libraries

In [200]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [201]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [202]:
%run Data_preprocessing.ipynb

0
0
0


In [203]:
len(X_train.columns)

7488

# Hyperparameters

In [204]:
input_size = 312 # 7488/24
sequence_length = 24
output_size = 2 # live or dead
num_layers = 2
hidden_size=256
learning_rate = 0.01
batch_size = 64
num_epochs = 2

In [205]:
7488/24

312.0

# Create RNN

In [254]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_epochs)
        # N * time_seq * features
        
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size).to(device)
        out, _ = self.rnn(x,h0)
        out = out.reshape(out.shape[0],-1)
        out = (F.softmax(self.fc(out), dim=1))
        print(out)
        return out

# Load data

In [255]:
class CusDatasetLoader(Dataset):
    def __init__(self,x,y):
        self.len = y.shape[0]
        self.x_data = x
        self.y_data = y
  
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        #return self.dataframe.iloc[index]
        return self.x_data[index], self.y_data[index]

In [256]:
def X_split(X):
    X_tensor = torch.from_numpy(np.array(X)).to(torch.float32)
    print(X_tensor.shape)
    X_tensor = torch.split(X_tensor,24,1)
    X_tensor = torch.stack(X_tensor)
    print(X_tensor.shape)
    return X_tensor

def y_tensor(y):
    y= torch.from_numpy(np.array(y)).to(torch.float32)
    return y

In [257]:
len(y_train_t1.iloc[:,-1])

16760

In [258]:
# X should be in smaples, 1, squence, rows
X1 = X_split(X_train)
X1 = X1.permute(1, 0, 2)
print(X1.shape)
y1 = y_tensor(y_train_t1.iloc[:,-1]).reshape(len(y_train_t1.iloc[:,-1]),1)
print(y1.shape)
train_datasets = CusDatasetLoader(X1, y1)
train_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)

torch.Size([16760, 7488])
torch.Size([312, 16760, 24])
torch.Size([16760, 312, 24])
torch.Size([16760, 1])


In [259]:
x,y = train_datasets[0]
print(x.shape)
print(y.shape)

torch.Size([312, 24])
torch.Size([1])


In [260]:
model = RNN(input_size, hidden_size, num_layers, output_size).to(device)

In [261]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

#

In [262]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.permute(0,2,1)
        print(data.shape)
        data = data.to(device=device)
        targets = targets.to(device=device)
        targets = torch.squeeze(targets, 1)
        print(targets.shape)
        
        # forward
        scores = model(data)
        print(scores.shape)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        
        

torch.Size([64, 24, 312])
torch.Size([64])
tensor(89)
torch.Size([])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

#

#

#

#

#

#

#